In [49]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [50]:
SCORE_DIR = "data/scores"

In [51]:
box_scores = os.listdir(SCORE_DIR)

In [52]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")] 

In [53]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [54]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score 

In [55]:
def read_stats (soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
    

In [75]:
base_cols = None
box_score = box_scores[0]

#for box_score in box_scores:
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")
    
    totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1, :]])
    totals.index = totals.index.str.lower()
    
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
    maxes.index = maxes.index.str.lower() + "_max"
    
    summary = pd.concat([totals, maxes])

    if base_cols is None:
        base_cols = list(summary.index.drop_duplicates(keep-"first"))
    



In [76]:
summary

mp          240.00
fg           36.00
fga         100.00
fg%           0.36
3p            7.00
             ...  
tov%_max     22.80
usg%_max     29.00
ortg_max    178.00
drtg_max    111.00
bpm_max       5.50
Length: 72, dtype: float64